In [1]:
from Bio import SeqIO

In [2]:
with open("GRCh37.p13_chr17.fna") as handle:
    for record in SeqIO.parse(handle, 'fasta'):
        seq_chr17 = str(record.seq)
        break

In [ ]:
WINDOW_SIZE = 2048

# Read the reference genome sequence of chromosome 17
with open("GRCh37.p13_chr17.fna", "rt") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        seq_chr17 = str(record.seq)
        break

def parse_sequences(pos, ref, alt):
    """
    Parse reference and variant sequences from the reference genome sequence.
    """
    p = pos - 1 # Convert to 0-indexed position
    full_seq = seq_chr17

    ref_seq_start = max(0, p - WINDOW_SIZE//2)
    ref_seq_end = min(len(full_seq), p + WINDOW_SIZE//2)
    ref_seq = seq_chr17[ref_seq_start:ref_seq_end]
    snv_pos_in_ref = min(WINDOW_SIZE//2, p)
    var_seq = ref_seq[:snv_pos_in_ref] + alt + ref_seq[snv_pos_in_ref+1:]

    # Sanity checks
    assert len(var_seq) == len(ref_seq)
    assert ref_seq[snv_pos_in_ref] == ref
    assert var_seq[snv_pos_in_ref] == alt

    return ref_seq, var_seq

# Parse sequences for the first variant
ref_seq, var_seq = parse_sequences(41276135, "T", "G")

Reference, SNV 0: ......
Variant, SNV 0:   ......


In [ ]:
import requests
import os
import json

# Replace with your actual API key
api_key = input("Paste the Run Key: ")

# API endpoint
url = os.getenv('URL', 'https://health.api.nvidia.com/v1/biology/arc/evo2-40b/forward')

# Headers
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json'
}

# Payload
payload = {
    "sequence": ref_seq,
    "output_layers": ["embedding_layer"],
}

# Make the request
response = requests.post(url, json=payload, headers=headers)

# Check the response
if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print('Error:', response.status_code, response.text)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)